In [ ]:
%pip install pandas matplotlib

In [ ]:
import pickle
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# util functions


def pivot_by_cond(df, params, x="num_node", value="user", cond=dict()):
    _df = df.query(" and ".join([f"{k} == {cond[k]}" for k in cond])) if cond else df

    def skip_elem(arr, elem):
        """Return a new array with the element removed."""
        return [e for e in arr if e != elem]

    return (
        _df.set_index(params)
        .reset_index(x)
        .pivot_table(index=x, columns=skip_elem(params, x), values=value)
    )


def load_results_from_pickle():
    with open("results.pickle", "rb") as f:
        dataset = pickle.load(f)
        all_params = dataset["__params__"]
        df = pd.DataFrame([dataset[k] for k in dataset if k != "__params__"])
        print(len(dataset), "results found")
        # removed keys to reduce params
        removed_keys = [k for k in df.columns if len(df[k].unique()) == 1]
        params = [k for k in all_params if k not in removed_keys]
        print("params: ", params)
        sim_names = [k for k in dataset.keys() if k != "__params__"]
        return df, params, dataset, sim_names


def load_logs_from_jsonl(sim_names):
    logs = {}
    for sim_name in sim_names:
        logs[sim_name] = pd.read_json(
            f"./results/{sim_name}.jsonl", orient="records", lines=True
        )
    return logs


In [ ]:
# load results and parameters from pickle
dataset = None
params = []
results = ["num_total_events", "user", "sys"]

df,params,dataset,sim_names = load_results_from_pickle()
print(df.dtypes)


In [ ]:
df[params]

In [ ]:
# you can specify the conditions to filter the dataframe
condition = {"purification_type": 1001, "connection_type": '"MIM"'}

pivot_by_cond(df,params=params, x="num_node", value="user", cond=condition).plot(
    figsize=(15, 15), ylabel="duration (s)"
)

In [ ]:
logs = load_logs_from_jsonl(sim_names)
sim_name = sim_names[1]
log = logs[sim_name]
durations = {}

def with_qubit_key(record):
    return (
        record["address"],
        record["qnic_type"],
        record["qnic_index"],
        record["qubit_index"],
    )


log["key"] = log[log["event_type"] == "QubitStateChange"].apply(with_qubit_key, axis=1)

for _, rec in log.iterrows():
    key = rec["key"]
    if key in durations:
        if rec["busy"]:
            durations[key]["last"] = rec["simtime"]
        else:
            last = durations[key]["last"]
            if last:
                durations[key]["usage"].append((last, rec["simtime"] - last))
            last = None

    else:
        durations[key] = {"last": rec["simtime"], "usage": []}

usages = []
for key in durations:
    d = durations[key]
    for us in d["usage"]:
        usages.append((str(key), us[0], us[1]))
usages = sorted(usages)
usages.reverse()
titles, begins, width = zip(*usages)
plt.subplots(figsize=(25, 22))
plt.xlim([0,0.0005])
plt.barh(y=titles, width=width, left=begins,linewidth=0.5, edgecolor="black")


In [ ]:
log["event_type"].unique()